In [21]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 39, Finished, Available, Finished)

# Automated ML
## Introduction

This notebook is automatically generated by the Fabric low-code AutoML wizard based on your selections. Whether you're building a regression model, a classifier, or another machine-learning solution, this tool simplifies the process by transforming your goals into executable code. You can easily modify any settings or code snippets to better align with your requirements.

### What is FLAML?

[FLAML (Fast and Lightweight Automated Machine Learning)](https://aka.ms/fabric-automl) is an open-source AutoML library designed to quickly and efficiently find the best machine learning models and hyperparameters. FLAML optimizes for speed, accuracy, and cost, making it an excellent choice for a wide range of machine-learning tasks.

### Steps in this notebook

1. **Load the data**: Import your dataset.
2. **Generate features**: Automatically transform and preprocess your data to improve model performance.
3. **Use AutoML to find your best model**: Use FLAML to automatically select the most suitable model and optimize its parameters.
4. **Save the final machine learning model**: Store the trained model for future use.
5. **Generate predictions**: Use the saved model to predict outcomes on new data.

> [!IMPORTANT]
> **The forecasting functionality is currently supported only on Pandas DataFrames.**
> **Automated ML is currently supported on Fabric Runtimes 1.2+ or any Fabric environment with Spark 3.4+.**


In [40]:
%pip install scikit-learn==1.5.1
# Cell 1: Run the installation
# %pip install "pandas<2.0"

StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 76, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



### Default notebook optimization

This cell configures the logging and warning settings to reduce unnecessary output and focus on critical information. It suppresses specific warnings and logs from the underlying libraries, ensuring a cleaner and more readable notebook experience.

In [41]:
import logging
import warnings
 
logging.getLogger('synapse.ml').setLevel(logging.CRITICAL)
logging.getLogger('mlflow.utils').setLevel(logging.CRITICAL)
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)

StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 78, Finished, Available, Finished)

## Step 1: Load the Data

This cell is responsible for importing the raw data from the specified source into the notebook environment. The data could come from various sources, such as a file or table in your lakehouse.

Once loaded, this data will serve as the input for subsequent steps, such as data transformation, model training, and evaluation.

In [42]:
import re
import pandas as pd
import numpy as np

df = spark.read.format("delta").load(
    "Tables/dbo/despesas_alucar_normalizada"
).cache()
# Transform to pandas according to the selected models
X = df.limit(100000).toPandas() # Use df.toPandas() to use all the data
X = X.rename(columns = lambda c:re.sub('[^A-Za-z0-9_]+', '_', c))  # Replace not supported characters in column name with underscore to avoid invalid character for model training and saving
target_col = re.sub('[^A-Za-z0-9_]+', '_', "Valor_Despesa")
X[target_col] = X[target_col].astype(str) \
                             .str.replace('R$', '', regex=False) \
                             .str.strip() \
                             .str.replace('.', '', regex=False) \
                             .str.replace(',', '.', regex=False)
X[target_col] = pd.to_numeric(X[target_col], errors='coerce')


StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 79, Finished, Available, Finished)

In [43]:
display(X)

StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 80, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 83d7ae51-89d8-409b-82f6-285da7dd4b47)

## Step 2: Generate features

Featurization is the process of transforming raw data into a format optimized for training a machine learning model. It ensures the model can access the most relevant information, significantly impacting its accuracy and performance.

This step applies various techniques to refine the data, enhance its quality, and make it compatible with the selected algorithms, helping the model learn patterns more effectively.

In [44]:
# Set Functions if needed for Featurization
def create_fillna_processor(
    df, mean_features=None, median_features=None, mode_features=None
):
    """
    Create a ColumnTransformer that fills missing values in a DataFrame using different strategies
    based on the skewness of the numerical features and the specified feature lists.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    mean_features (list, optional): List of features to impute using the mean strategy. Defaults to None.
    median_features (list, optional): List of features to impute using the median strategy. Defaults to None.
    mode_features (list, optional): List of features to impute using the mode strategy. Defaults to None.

    Returns:
    ColumnTransformer: A fitted ColumnTransformer that can be used to transform the DataFrame.
    list: List of all features supported by SimpleImputer in the DataFrame.
    list: List of datetime features in the DataFrame.
    """
    if mean_features is None:
        mean_features = []
    if median_features is None:
        median_features = []
    if mode_features is None:
        mode_features = []
    all_features = mean_features + median_features + mode_features
    # Group features by their imputation needs
    mean_features = [
        col
        for col in df.select_dtypes(include=["number"]).columns
        if df[col].skew(skipna=True) <= 1 and col not in all_features
    ] + mean_features
    median_features = [
        col
        for col in df.select_dtypes(include=["number"]).columns
        if df[col].skew(skipna=True) > 1 and col not in all_features
    ] + median_features
    all_features = mean_features + median_features
    datetime_features = df.select_dtypes(include=["datetime"]).columns.tolist()
    mode_features = [col for col in df.columns.tolist() if col not in all_features + datetime_features]

    transformers = []

    if mean_features:
        transformers.append(
            ("mean_imputer", SimpleImputer(strategy="mean"), mean_features)
        )
    if median_features:
        transformers.append(
            ("median_imputer", SimpleImputer(strategy="median"), median_features)
        )
    if mode_features:
        transformers.append(
            ("mode_imputer", SimpleImputer(strategy="most_frequent"), mode_features)
        )

    column_transformer = ColumnTransformer(transformers=transformers)
    all_features = mean_features + median_features + mode_features

    return column_transformer.fit(df), all_features, datetime_features


def fillna(df, processor, all_features, datetime_features):
    """
    Fill missing values in a DataFrame using a specified processor and mode imputation.

    Parameters:
    df (pd.DataFrame): The input DataFrame with missing values.
    processor (object): An object with a `transform` method that processes the DataFrame.
    all_features (list): List of all features supported by SimpleImputer in the DataFrame.
    datetime_features (list): List of datetime features in the DataFrame.

    Returns:
    pd.DataFrame: A DataFrame with missing values filled.
    """
    filled_array = processor.transform(df)
    filled_df = pd.DataFrame(filled_array, columns=all_features)
    if datetime_features:
        datetime_data = df[datetime_features]
        datetime_data.ffill()
        filled_df = pd.concat([datetime_data, filled_df], axis=1)
    for col in df.columns:
        filled_df[col].fillna(filled_df[col].mode()[0], inplace=True)

    return filled_df


StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 81, Finished, Available, Finished)

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


time_col = "Data_Referencia"
ts_col = X.pop(time_col)
X.insert(0, time_col, ts_col.apply(lambda x: np.datetime64(x, "ns")))

cast_type = [['Valor_Despesa', 'float']]
for col, coltype in cast_type:
    col = re.sub('[^A-Za-z0-9_]+', '_', col)
    X[col] = X[col].astype(coltype)

# convert object type to nearest dtype
X = X.convert_dtypes()
X = X.dropna(axis=1, how='all')

# select columns for model training
X = X.select_dtypes(include=['number', 'datetime', 'category'])

from sklearn.model_selection import train_test_split

# You may need to update the test_size based on your scenario
X_train, X_test = train_test_split(X, test_size=int(X.shape[0] / 3 * 0.2) * 3, shuffle=False, random_state=41)

mean_features, median_features, mode_features = [], [], []
 
preprocessor, all_features, datetime_features = create_fillna_processor(X_train, mean_features, median_features, mode_features)
X_train = fillna(X_train, preprocessor, all_features, datetime_features)
X_test = fillna(X_test, preprocessor, all_features, datetime_features)
 
y_train = X_train.pop(target_col)
y_test = X_test.pop(target_col)

display(X_train[:10])


StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 82, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e6b24fa9-30ae-427c-91e8-4279e3ccc3bb)

## Step 3: Use AutoML to find your best model

We will now use FLAML's AutoML to automatically find the best machine learning model for our data. AutoML (Automated Machine Learning) simplifies the model selection process by automatically testing and tuning various algorithms and configurations, helping us quickly identify the most effective model with minimal manual effort.

### Tracking results with experiments in Fabric

Experiments in Fabric let you track the results of your AutoML process, providing a comprehensive view of all the metrics and parameters from your trials.

In [46]:
# MLFlow Logging Related

import mlflow

mlflow.autolog(exclusive=False)
mlflow.set_experiment("Notebook1-AutoMLExperiment-DespesaAlucar")


StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 83, Finished, Available, Finished)

2025/06/26 01:24:43 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


<Experiment: artifact_location='', creation_time=1750899968522, experiment_id='30722df6-b617-48af-b63f-dbd61c6c2a4a', last_update_time=None, lifecycle_stage='active', name='Notebook1-AutoMLExperiment-DespesaAlucar', tags={}>

#### Configure the AutoML trial and settings

These configurations are driven by the AutoML mode and task selected in the wizard. For example, if you select "quick prototype", you'll see a setting for time budget.

In [47]:
# Import the AutoML class from the FLAML package
import flaml
from flaml import AutoML

# Define AutoML settings
settings = {
    "time_budget": 120, # Total running time in seconds
    "estimator_list": ['lgbm', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'prophet'],  # estimator_list for spark35 forecasting 
    "task": "ts_forecast",  # Task type 
    "log_file_name": "flaml_experiment.log",  # FLAML log file
    "max_iter": 10, 
    "force_cancel": True, 
    "seed": 41 , # Random seed 
    "mlflow_exp_name": "Notebook1-AutoMLExperiment-DespesaAlucar",  # MLflow experiment name
    "use_spark": True, # whether to use Spark for distributed training
    "n_concurrent_trials": 3,  # the maximum number of concurrent trials 
    "verbose": 1,  
    "featurization": "auto", 
}

if flaml.__version__ > "2.3.3":
    settings["entrypoint"] = "low-code"

# Create an AutoML instance
automl = AutoML(**settings)


StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 84, Finished, Available, Finished)

2025/06/26 01:24:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.
2025/06/26 01:24:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/06/26 01:24:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/06/26 01:24:56 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2025/06/26 01:24:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.


#### Run the AutoML trial

Run the AutoML trial, with all trials being tracked as experiment runs. The trial is performed on the processed dataset, using the `Exited` variable as the target, and applying the defined configurations for optimal model selection.

In [33]:
# Cell 1: Run the installation
# %pip install "pandas<2.0"

StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 63, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [48]:
with mlflow.start_run(nested=True, run_name="Notebook1-AutoMLModel"):
    automl.fit(
        X_train=X_train, 
        y_train=y_train,  # target column of the training data 
        period=3, 
    )

StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, 85, Finished, Available, Finished)

AssertionError: Only time series of regular frequency are currently supported.

## Step 4: Save the final machine learning model

Upon completing the AutoML trial, you can now save the final, tuned model as an ML model in Fabric.

In [ ]:
model_path = f"runs:/{automl.best_run_id}/model"

# Register the model to the MLflow registry
registered_model = mlflow.register_model(model_uri=model_path, name="Notebook1-AutoMLModel")

# Print the registered model's name and version
print(f"Model '{registered_model.name}' version {registered_model.version} registered successfully.")

StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, -1, Cancelled, , Cancelled)

## Step 5: Generate predictions

1. Generate predictions.

In [ ]:
loaded_model_pred = automl.predict(X_test)
print('Predicted labels', loaded_model_pred)


StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, -1, Cancelled, , Cancelled)

2. Save the predictions to a table.

In [ ]:
from pyspark.sql.types import FloatType
predictions = spark.createDataFrame(loaded_model_pred, FloatType())
saved_name = "despesas_alucar_normalizada_predictions".replace(".", "_")
predictions.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(f"Tables/{saved_name}")

StatementMeta(, 343ca31d-b156-47c2-b805-875d5f093cb9, -1, Cancelled, , Cancelled)